Takes around 20 min to run

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# *Reading in Full CSV as a DF (2min)*

In [ ]:
import pandas as pd
import glob
import os
import numpy as np

# Full NCAA Dataset - 1:48 min
df = pd.read_csv("/content/drive/MyDrive/USC Baseball/Datasets/Updated22_23_NCAA_Dataset.csv")

<ipython-input-3-9eb627f2f207>:7: DtypeWarning: Columns (27,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/USC Baseball/Datasets/Updated22_23_NCAA_Dataset.csv")


## Read in Team Name Conversion


In [ ]:
teamnameDF = pd.read_csv("/content/drive/Shareddrives/USC Baseball/TeamNameConversions.csv")

## Team Name conversion Function

In [ ]:
def convertTeamname(trackmanTeam, teamnameDF) -> str:
  mask = teamnameDF['TrackmanName'].astype(str).str.contains(trackmanTeam)
  if mask.any():
    return teamnameDF.loc[mask, 'SchoolName'].values[0]
  else:
    return trackmanTeam

# **Transfer Concat**

In [ ]:
df2 = df.copy()

df2.drop_duplicates()

# this line removes all extra rows at the bottom
df2.dropna(how='all', inplace=True)
cleandf = df2.dropna(subset=['RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak'])
pitchdf = cleandf.copy()
batdf = cleandf.copy()


#This will ignore all DeprecationWarning warnings in your code.
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# Open up workbook
workbook = gc.open('Transfer Portal 2024')
# obtain worksheet titles from workbook
sheet_titles = []
for sheet in workbook.worksheets():
  sheet_titles.append(sheet.title)
sheet_titles

['Entire Transfer Portal',
 'D1Baseball List',
 'Players We Like',
 'Batter Stats',
 'Batter Metrics',
 'Pitcher Stats',
 'Pitcher Metrics']

In [ ]:
# Get_all_values from sheets and store in dictionary
dict_of_sheets = {}
for sheet_title in sheet_titles:
  sheet = workbook.worksheet(sheet_title)
  values = sheet.get_all_values()
  dict_of_sheets[sheet_title] = values
# To confirm all sheets made it into the dictionary
dict_of_sheets.keys()

dict_keys(['Entire Transfer Portal', 'D1Baseball List', 'Players We Like', 'Batter Stats', 'Batter Metrics', 'Pitcher Stats', 'Pitcher Metrics'])

In [ ]:
transfer_df = pd.DataFrame.from_records(dict_of_sheets.get('Entire Transfer Portal'))
new_header = transfer_df.iloc[0]  # Get the first row for the header
transfer_df = transfer_df[1:]  # Take the data less the header row
transfer_df.columns = new_header  # Set the header row as the DataFrame header

transfer_list = transfer_df['Trackman Name'].unique().tolist()
pitcher_list = transfer_df[transfer_df['Position'].str.contains('P', na=False)]['Trackman Name'].unique().tolist()
batter_list = transfer_df[transfer_df['Position'].str.contains('P-', na=False) | ~transfer_df['Position'].str.contains('P', na=False)]['Trackman Name'].unique().tolist()

# # names of all transfers (pitchers and batters)
# transfer_df = pd.read_csv('/content/drive/Shareddrives/USC Baseball/Transfer Portal Information/2024 Transfer Portal/Transfers2024.csv')
# # list of names
# transfer_list = transfer_df['Trackman Name'].unique().tolist()

In [ ]:
# pitcher transfers

# creates a Boolean series called pitcher by checking whether each value in the 'Pitcher' column of the DataFrame df is present in the transfer_list.
# resulting series will have True for rows where the 'Pitcher' value is in the transfer_list, and False otherwise.
pitcher = pitchdf['Pitcher'].isin(pitcher_list)
# # makes a dataframe that only contains rows of pitchers that are in the transfer list
df_pitcher = pitchdf[pitcher]
# pitcher_list = df_pitcher['Pitcher'].unique()

In [ ]:
# hitter transfers

# creates a Boolean series called pitcher by checking whether each value in the 'Batter' column of the DataFrame df is present in the transfer_list.
# resulting series will have True for rows where the 'Batter' value is in the transfer_list, and False otherwise.
batter = batdf['Batter'].isin(batter_list)

# # makes a dataframe that only contains rows of batters that are in the transfer list
df_batter = batdf[batter]
# batter_list = df_batter['Batter'].unique()


# ***Read in Batter and Pitcher Data from Trackman***

## Pitcher Data

In [ ]:
# PITCHER TRACKMAN DATA

import pandas as pd
import numpy as np

# Define constants and initialize the final DataFrame
division_list = ["D1", "D2", "D3", "JUCO"]
pitcher_trackman = pd.DataFrame(columns=[
    'Position', 'Division', 'School', 'Pitcher', 'PitchType', 'Sample Size', 'Velo', 'Spin Rate', 'Spin Axis',
    'IVB', 'HB', 'Extension', 'VAA', 'HAA', 'Strike %', 'Whiff %', 'Zone Whiff %', 'Chase %', 'Stuff+', 'Location+'
])

def calculate_stats(pitcher_df, pitch_type):
    stats = {}
    filtered_df = pitcher_df[pitcher_df['TaggedPitchType'] == pitch_type]

    if filtered_df.empty:
        return None

    stats['Sample Size'] = len(filtered_df)
    stats['Velo'] = filtered_df['RelSpeed'].mean()
    stats['Spin Rate'] = filtered_df['SpinRate'].mean()
    stats['Spin Axis'] = filtered_df['SpinAxis'].mean()
    stats['IVB'] = filtered_df['InducedVertBreak'].mean()
    stats['HB'] = filtered_df['HorzBreak'].mean()
    stats['Extension'] = filtered_df['Extension'].mean()
    stats['VAA'] = filtered_df['VertApprAngle'].mean()
    stats['HAA'] = filtered_df['HorzApprAngle'].mean()

    strike_swing = filtered_df['PitchCall'].isin(['StrikeSwinging'])
    swing = filtered_df['PitchCall'].isin(['StrikeSwinging', 'InPlay', 'FoulBall'])

    stats['Whiff %'] = strike_swing.sum() / swing.sum() if swing.sum() > 0 else 0
    stats['Strike %'] = swing.sum() / stats['Sample Size']

    in_zone = filtered_df[(filtered_df['PlateLocSide'].between(-0.951, 0.951)) & (filtered_df['PlateLocHeight'].between(1.5, 3.6))]
    out_zone = filtered_df[~((filtered_df['PlateLocSide'].between(-0.951, 0.951)) & (filtered_df['PlateLocHeight'].between(1.5, 3.6)))]

    z_swing = in_zone['PitchCall'].isin(['StrikeSwinging', 'InPlay', 'FoulBall']).sum()
    z_whiff = in_zone['PitchCall'].isin(['StrikeSwinging']).sum()
    o_pitch = len(out_zone)
    o_swing = out_zone['PitchCall'].isin(['StrikeSwinging', 'InPlay', 'FoulBall']).sum()

    stats['Zone Whiff %'] = z_whiff / z_swing if z_swing > 0 else 0
    stats['Chase %'] = o_swing / o_pitch if o_pitch > 0 else 0

    return stats

# Loop through each division
for division in division_list:
    division_df = df_pitcher[df_pitcher['Level'] == division]

    # Loop through each pitcher
    for pitcher in pitcher_list:
        pitcher_df = division_df[division_df['Pitcher'] == pitcher]

        # Get list of teams and unique pitches
        pitcher_teams_list = pitcher_df['PitcherTeam'].unique().tolist()

        for team in pitcher_teams_list:
            team_df = pitcher_df[pitcher_df['PitcherTeam'] == team]
            unique_pitch_types_list = team_df['TaggedPitchType'].unique().tolist()

            for pitch_type in unique_pitch_types_list:
                if pitch_type == "Undefined":
                    continue

                stats = calculate_stats(team_df, pitch_type)
                if stats is None:
                    continue

                pitch_hand = "LHP" if (team_df['PitcherThrows'] == "Left").all() else "RHP"
                new_row = {
                    'Position': pitch_hand,
                    'Division': division,
                    'School': convertTeamname(team, teamnameDF),
                    'Pitcher': pitcher,
                    'PitchType': pitch_type,
                    'Sample Size': stats['Sample Size'],
                    'Velo': stats['Velo'],
                    'Spin Rate': stats['Spin Rate'],
                    'Spin Axis': stats['Spin Axis'],
                    'IVB': stats['IVB'],
                    'HB': stats['HB'],
                    'Extension': stats['Extension'],
                    'VAA': stats['VAA'],
                    'HAA': stats['HAA'],
                    'Strike %': stats['Strike %'],
                    'Whiff %': stats['Whiff %'],
                    'Zone Whiff %': stats['Zone Whiff %'],
                    'Chase %': stats['Chase %'],
                    'Stuff+': 0.0,      # placeholder, to be calculated
                    'Location+': 0.0  # placeholder, to be calculated

                }
                new_row_df = pd.DataFrame([new_row])
                pitcher_trackman = pd.concat([pitcher_trackman, new_row_df], ignore_index=True)


## Hitter Data

In [ ]:
# HITTER TRACKMAN DATA

import pandas as pd
import numpy as np

# Define constants and initialize the final DataFrame
division_list = ["D1", "D2", "D3", "JUCO"]
batter_trackman = pd.DataFrame(columns=[
    'Division', 'School', 'Batter', 'Sample Size', 'AVG Exit Velo', '90th% EV', 'MAX Exit Velo',
    '% EV > 95 (Hard Hit Ball %)', '25th% Launch Angle', 'Median Launch Angle', '75th% Launch Angle',
    '% Angle btwn 10-25 (Ideal LA %)', 'SwingSDS', 'TakeSDS', 'Whiff %', 'O-Whiff %', 'Z-Whiff %',
    'Swing %', 'O-Swing %', 'Z-Swing %', 'Contact %', 'O-Contact %', 'Z-Contact %', 'xBA', 'xBABIP',
    'xSLG', 'xwOBA', 'xISO'
])

def calculate_batter_stats(filter_df, cleanBIP):
    stats = {}
    stats['AVG Exit Velo'] = cleanBIP['ExitSpeed'].mean()
    stats['90th% EV'] = cleanBIP['ExitSpeed'].quantile(0.9)
    stats['MAX Exit Velo'] = cleanBIP['ExitSpeed'].max()
    stats['% EV > 95 (Hard Hit Ball %)'] = (cleanBIP['ExitSpeed'] > 95).sum()
    stats['25th% Launch Angle'] = cleanBIP['Angle'].quantile(0.25)
    stats['Median Launch Angle'] = cleanBIP['Angle'].median()
    stats['75th% Launch Angle'] = cleanBIP['Angle'].quantile(0.75)
    stats['% Angle btwn 10-25 (Ideal LA %)'] = ((cleanBIP['Angle'] >= 10) & (cleanBIP['Angle'] <= 25)).sum()
    stats['Sample Size'] = len(cleanBIP)
    stats['Hard Hit Ball %'] = stats['% EV > 95 (Hard Hit Ball %)'] / stats['Sample Size'] if stats['Sample Size'] > 0 else 0
    stats['Ideal LA %'] = stats['% Angle btwn 10-25 (Ideal LA %)'] / stats['Sample Size'] if stats['Sample Size'] > 0 else 0

    whiff_count = filter_df['PitchCall'].isin(['StrikeSwinging']).sum()
    swing_count = filter_df['PitchCall'].isin(['StrikeSwinging', 'InPlay', 'FoulBall']).sum()
    pitch_count = filter_df['PitchCall'].isin([
        'BallCalled', 'StrikeCalled', 'HitByPitch', 'BallinDirt',
        'StrikeSwinging', 'InPlay', 'FoulBall'
    ]).sum()
    contact_count = filter_df['PitchCall'].isin(['InPlay', 'FoulBall']).sum()

    stats['Whiff %'] = whiff_count / swing_count if swing_count > 0 else 0
    stats['Swing %'] = swing_count / pitch_count if pitch_count > 0 else 0
    stats['Contact %'] = contact_count / swing_count if swing_count > 0 else 0

    z_pitch_count = filter_df[((filter_df['PlateLocSide'] >= -0.951) & (filter_df['PlateLocSide'] <= 0.951)) &
                              ((filter_df['PlateLocHeight'] >= 1.5) & (filter_df['PlateLocHeight'] <= 3.6))].shape[0]
    z_swing_count = filter_df[(filter_df['PlateLocSide'].between(-0.951, 0.951)) &
                              (filter_df['PlateLocHeight'].between(1.5, 3.6))]['PitchCall'].isin([
                                  'StrikeSwinging', 'InPlay', 'FoulBall'
                              ]).sum()
    z_whiff_count = filter_df[(filter_df['PlateLocSide'].between(-0.951, 0.951)) &
                              (filter_df['PlateLocHeight'].between(1.5, 3.6))]['PitchCall'].isin(['StrikeSwinging']).sum()
    z_contact_count = filter_df[(filter_df['PlateLocSide'].between(-0.951, 0.951)) &
                                (filter_df['PlateLocHeight'].between(1.5, 3.6))]['PitchCall'].isin(['InPlay', 'FoulBall']).sum()

    o_pitch_count = filter_df[~((filter_df['PlateLocSide'].between(-0.951, 0.951)) &
                                (filter_df['PlateLocHeight'].between(1.5, 3.6)))].shape[0]
    o_swing_count = filter_df[~((filter_df['PlateLocSide'].between(-0.951, 0.951)) &
                                (filter_df['PlateLocHeight'].between(1.5, 3.6)))]['PitchCall'].isin([
                                    'StrikeSwinging', 'InPlay', 'FoulBall'
                                ]).sum()
    o_whiff_count = filter_df[~((filter_df['PlateLocSide'].between(-0.951, 0.951)) &
                                (filter_df['PlateLocHeight'].between(1.5, 3.6)))]['PitchCall'].isin(['StrikeSwinging']).sum()
    o_contact_count = filter_df[~((filter_df['PlateLocSide'].between(-0.951, 0.951)) &
                                  (filter_df['PlateLocHeight'].between(1.5, 3.6)))]['PitchCall'].isin(['InPlay', 'FoulBall']).sum()

    stats['Z-Whiff %'] = z_whiff_count / z_swing_count if z_swing_count > 0 else 0
    stats['Z-Swing %'] = z_swing_count / z_pitch_count if z_pitch_count > 0 else 0
    stats['Z-Contact %'] = z_contact_count / z_swing_count if z_swing_count > 0 else 0

    stats['O-Whiff %'] = o_whiff_count / o_swing_count if o_swing_count > 0 else 0
    stats['O-Swing %'] = o_swing_count / o_pitch_count if o_pitch_count > 0 else 0
    stats['O-Contact %'] = o_contact_count / o_swing_count if o_swing_count > 0 else 0

    return stats

# Loop through each division
for division in division_list:
    division_df = df_batter[df_batter['Level'] == division]

    # Loop through each batter
    for batter in batter_list:
        batter_df = division_df[division_df['Batter'] == batter]

        # Get list of teams
        batter_teams_list = batter_df['BatterTeam'].unique().tolist()

        for team in batter_teams_list:
            filter_df = batter_df[batter_df['BatterTeam'] == team]

            # Create a cleaned dataframe only consisting of balls in play with complete data
            BIP_df = filter_df[filter_df['PitchCall'] == "InPlay"]
            cleanBIP = BIP_df.dropna(subset=['ExitSpeed', 'Angle'])

            if cleanBIP.empty:
                continue

            stats = calculate_batter_stats(filter_df, cleanBIP)

            new_row = {
                'Division': division,
                'School': convertTeamname(team, teamnameDF),
                'Batter': batter,
                'Sample Size': stats['Sample Size'],
                'AVG Exit Velo': stats['AVG Exit Velo'],
                '90th% EV': stats['90th% EV'],
                'MAX Exit Velo': stats['MAX Exit Velo'],
                '% EV > 95 (Hard Hit Ball %)': stats['Hard Hit Ball %'],
                '25th% Launch Angle': stats['25th% Launch Angle'],
                'Median Launch Angle': stats['Median Launch Angle'],
                '75th% Launch Angle': stats['75th% Launch Angle'],
                '% Angle btwn 10-25 (Ideal LA %)': stats['Ideal LA %'],
                'SwingSDS': 0.0,  # fill later
                'TakeSDS': 0.0, # fill later
                'Whiff %': stats['Whiff %'],
                'O-Whiff %': stats['O-Whiff %'],
                'Z-Whiff %': stats['Z-Whiff %'],
                'Swing %': stats['Swing %'],
                'O-Swing %': stats['O-Swing %'],
                'Z-Swing %': stats['Z-Swing %'],
                'Contact %': stats['Contact %'],
                'O-Contact %': stats['O-Contact %'],
                'Z-Contact %': stats['Z-Contact %'],
                'xBA': 0.0, # fill later
                'xBABIP': 0.0,  # fill later
                'xSLG': 0.0,  # fill later
                'xwOBA': 0.0, # fill later
                'xISO': 0.0,  # fill later
            }

            new_row_df = pd.DataFrame([new_row])
            batter_trackman = pd.concat([batter_trackman, new_row_df], ignore_index=True)


# ***The following code is used to analyze a hitter's Swing and Take Decisions (SDS)* (Batters)**

In [ ]:
# HOLDS FUNCTIONS THAT CALCUALTE THE SDS+ OF EACH Player

import joblib
import numpy as np
from typing import Tuple
from scipy.stats import norm
import statistics
import pandas as pd
from pandas import DataFrame

df3 = df_batter.copy()

# Load models once to avoid repeated imports
contactProb = joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/SDS+/SDS+ Models/ContactProbModel.joblib')
swingProb = joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/SDS+/SDS+ Models/SwingProbModel.joblib')
strikeProb = joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/SDS+/SDS+ Models/StrikeProbModel.joblib')
hhbProb = joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/SDS+/SDS+ Models/HardHitProbModel.joblib')

# Utility function to calculate SDS+
def calculate_SDS_plus(sds_list: list, mean: float, std: float, min_val: float) -> float:
    avg_sds = np.mean(sds_list)
    sds_plus = (((avg_sds - min_val) - (mean - min_val)) / std) * 100 + 100
    return sds_plus

# Function to calculate the Swing SDS+ of each player
def swingSDS(filter_swing: DataFrame) -> float:
    cleanedSwing = filter_swing[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'PlateLocHeight', 'PlateLocSide', 'VertApprAngle', 'HorzApprAngle']]
    strikeSwing = filter_swing[['PlateLocHeight', 'PlateLocSide', 'pitchType', 'RelHeight', 'RelSide', 'hitterHand']]

    swingContact = contactProb.predict_proba(cleanedSwing)[:, 1]
    swingSwing = swingProb.predict_proba(cleanedSwing)[:, 1]
    swingStrike = strikeProb.predict_proba(strikeSwing)[:, 1]
    swingHHB = hhbProb.predict_proba(cleanedSwing)[:, 1]

    swingSDS_list = [(swingStrike[i] - (1 - swingStrike[i])) + swingSwing[i] + swingHHB[i] for i in range(len(cleanedSwing))]

    swingSDSMean = 1.8926646183979023
    swingSDSSTD = 0.9558850380546416
    swingSDSMin = -1.0

    return calculate_SDS_plus(swingSDS_list, swingSDSMean, swingSDSSTD, swingSDSMin)

# Function to calculate the Take SDS+ of each player
def takeSDS(filter_take: DataFrame) -> float:
    cleanedTake = filter_take[['RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'PlateLocHeight', 'PlateLocSide', 'VertApprAngle', 'HorzApprAngle']]
    strikeTake = filter_take[['PlateLocHeight', 'PlateLocSide', 'pitchType', 'RelHeight', 'RelSide', 'hitterHand']]

    takeContact = contactProb.predict_proba(cleanedTake)[:, 1]
    takeSwing = swingProb.predict_proba(cleanedTake)[:, 1]
    takeStrike = strikeProb.predict_proba(strikeTake)[:, 1]
    takeHHB = hhbProb.predict_proba(cleanedTake)[:, 1]

    takeSDS_list = [((1 - takeStrike[i]) - takeStrike[i]) - takeSwing[i] - takeHHB[i] for i in range(len(cleanedTake))]

    takeSDSMean = -0.07691706404188041
    takeSDSSTD = 1.1273762539917347
    takeSDSMin = -2.966666666666667

    return calculate_SDS_plus(takeSDS_list, takeSDSMean, takeSDSSTD, takeSDSMin)


In [ ]:
import numpy as np
import pandas as pd

# Drop rows where all elements are NaN
df3.dropna(how='all', inplace=True)

# Drop rows that don't have the needed values for specific columns
SDScleandf = df3.dropna(subset=['RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'PlateLocHeight', 'PlateLocSide', 'VertApprAngle', 'HorzApprAngle'])

# Encode pitch type
pitch_conditions = [
    SDScleandf['TaggedPitchType'].str.contains('Fastball|FourSeamFastBall|TwoSeamFastBall|OneSeamFastBall|Sinker', na=False),
    SDScleandf['TaggedPitchType'].str.contains('Slider|Cutter', na=False),
    SDScleandf['TaggedPitchType'].str.contains('ChangeUp|Splitter', na=False),
    SDScleandf['TaggedPitchType'].str.contains('Curveball', na=False)
]
pitch_values = [0, 1, 2, 3]
SDScleandf['pitchType'] = np.select(pitch_conditions, pitch_values, default=4)

# Encode batting side
bat_conditions = [
    SDScleandf['BatterSide'] == 'Right',
    SDScleandf['BatterSide'] == 'Left'
]
bat_values = [0, 1]
SDScleandf['hitterHand'] = np.select(bat_conditions, bat_values)

# Separate into swings & takes
swingdf = SDScleandf[SDScleandf['PitchCall'].str.contains("InPlay|StrikeSwinging|FoulBall", na=False)]
takedf = SDScleandf[SDScleandf['PitchCall'].str.contains("BallCalled|StrikeCalled|BallinDirt", na=False)]

# Iterate over divisions, batters, and teams to calculate ratings
spot = 0
for division in division_list:
    division_swing = swingdf[swingdf['Level'] == division]
    division_take = takedf[takedf['Level'] == division]

    for batter in batter_list:
        batter_swing = division_swing[division_swing['Batter'] == batter]
        batter_take = division_take[division_take['Batter'] == batter]

        batter_teams = np.union1d(batter_swing['BatterTeam'].unique(), batter_take['BatterTeam'].unique())

        for team in batter_teams:
            filter_swing = batter_swing[batter_swing['BatterTeam'] == team]
            filter_take = batter_take[batter_take['BatterTeam'] == team]

            swingSize = filter_swing.shape[0]
            takeSize = filter_take.shape[0]

            swingRating = swingSDS(filter_swing) if swingSize > 0 else ""
            takeRating = takeSDS(filter_take) if takeSize > 0 else ""

            batter_trackman.at[spot, 'SwingSDS'] = swingRating
            batter_trackman.at[spot, 'TakeSDS'] = takeRating

            spot += 1


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted with

# ***The following code is used to calculates a Pitcher's Location+ and Stuff+* (Pitchers)**

## Calculates Pitcher's Location+ Rating

In [ ]:
# HOLDS FUNCTIONS THAT CALCUALTE THE LOCATION+ OF EACH PITCH
import joblib
import pandas as pd
from pandas import DataFrame
import numpy as np

# Load all models once
locationModels = {
    'FB': {
        "StrikeProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Location+/Location+ Models/FBStrikeProbModel.joblib'),
        "HardHitProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Location+/Location+ Models/FBHardHitProbModel.joblib')
    },
    'SL': {
        "StrikeProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Location+/Location+ Models/SLStrikeProbModel.joblib'),
        "HardHitProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Location+/Location+ Models/SLHardHitProbModel.joblib')
    },
    'CH': {
        "StrikeProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Location+/Location+ Models/CHStrikeProbModel.joblib'),
        "HardHitProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Location+/Location+ Models/CHHardHitProbModel.joblib')
    },
    'CU': {
        "StrikeProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Location+/Location+ Models/CUStrikeProbModel.joblib'),
        "HardHitProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Location+/Location+ Models/CUHardHitProbModel.joblib')
    },
    'Other': {
        "StrikeProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Location+/Location+ Models/OtherStrikeProbModel.joblib'),
        "HardHitProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Location+/Location+ Models/OtherHardHitProbModel.joblib')
    }
}
test = {
    'FB': "testFB",
    'SL': "testSL",
    'CH': "testCH",
    'CU': "testCU",
    'Other': "testOther"
}

# Location+ information dictionary
location_info = {
    'FB': {"Mean": 0.406058499550797, "STD": 0.3532781763890214, "Min": -0.45},
    'SL': {"Mean": 0.34960684689716576, "STD": 0.3533850412263207, "Min": -0.45},
    'CH': {"Mean": 0.3353856241831144, "STD": 0.3451946132850606, "Min": -0.45},
    'CU': {"Mean": 0.3088163519897403, "STD": 0.35590971460457, "Min": -0.45},
    'Other': {"Mean": 0.2400966183574879, "STD": 0.35960819674468686, "Min": -0.45}
}

def calculate_location_rating(cleaned_data: DataFrame, model_key: str) -> float:
    # Apply models to datasets
    # print("Location model key: " + model_key)

    if model_key not in locationModels:
      print("Invalid model key: " + model_key)

    location_model = locationModels[model_key]

    strike_prob_model = location_model["StrikeProb"]
    hard_hit_prob_model = location_model["HardHitProb"]

    strike_probs = strike_prob_model.predict_proba(cleaned_data)[:, 1]  # Strike probability model
    hard_hit_probs = hard_hit_prob_model.predict_proba(cleaned_data)[:, 1]  # Hard hit probability model

    # Calculate Location Ratings
    location_ratings = strike_probs - (0.45 * hard_hit_probs)

    # Location+ information
    info = location_info[model_key]
    # loc_avg = np.mean(location_ratings)
    # loc_plus = (((loc_avg - info["Min"]) - (info["Mean"] - info["Min"])) / info["STD"]) * 100 + 100
    loc_avg = np.median(location_ratings)
    loc_plus = (loc_avg/info["Mean"])*100

    return loc_plus

# Individual pitch type functions
def fb_location(cleanedFB: DataFrame) -> float:
    return calculate_location_rating(cleanedFB, "FB")

def sl_location(cleanedSL: DataFrame) -> float:
    return calculate_location_rating(cleanedSL, "SL")

def ch_location(cleanedCH: DataFrame) -> float:
    return calculate_location_rating(cleanedCH, "CH")

def cu_location(cleanedCU: DataFrame) -> float:
    return calculate_location_rating(cleanedCU, "CU")

def other_location(cleanedOther: DataFrame) -> float:
    return calculate_location_rating(cleanedOther, "Other")


## Calculates a Pitcher's Stuff+

In [ ]:
# HOLDS FUNCTIONS THAT CALCUALTE THE STUFF+ OF EACH PITCH
import joblib
import pandas as pd
from pandas import DataFrame
import numpy as np

# Load all models once
stuffModels = {
    'Fastball': {
        "WhiffProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Stuff+/Stuff+ Models/FastballWhiffModel.joblib')
    },
    'Breaking': {
        "WhiffProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Stuff+/Stuff+ Models/BreakingWhiffModel.joblib')
    },
    'Offspeed': {
        "WhiffProb": joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/Stuff+/Stuff+ Models/OffspeedWhiffModel.joblib')
    }
}

# Stuff+ information dictionary
stuff_info = {
    'Fastball': {"Median": 0.23076923076923000},
    'Breaking': {"Median": 0.27000000000},
    'Offspeed': {"Median": 0.3100000}
}

def calculate_stuff_rating(cleaned_data: DataFrame, model_key: str) -> float:
    # Apply models to datasets

    if model_key not in stuffModels:
      print("Invalid model key: " + model_key)

    stuff_model = stuffModels[model_key]
    whiff_model = stuff_model["WhiffProb"]
    whiff_probs = whiff_model.predict_proba(cleaned_data)[:, 1]  # whiff probability model

    # Calculate Stuff Ratings
    stuff_ratings = whiff_probs

    # Stuff+ information
    info = stuff_info[model_key]
    loc_avg = np.median(stuff_ratings)
    loc_plus = (loc_avg/info["Median"])*100

    return loc_plus

# Individual pitch type functions
def fastball_stuff(cleanedFB: DataFrame) -> float:
    return calculate_stuff_rating(cleanedFB, "Fastball")

def breaking_stuff(cleanedSL: DataFrame) -> float:
    return calculate_stuff_rating(cleanedSL, "Breaking")

def offspeed_stuff(cleanedCH: DataFrame) -> float:
    return calculate_stuff_rating(cleanedCH, "Offspeed")


## Script to compute Location+ and Stuff+

In [ ]:
# GOES THROUGH ENTIRE PITCHER DF AGAIN AND CALCULATES LOCATION+ RATING
import pandas as pd
import numpy as np
from scipy.stats import norm

# Precompute conditions for pitch type and handedness classification
pitch_type_conditions = [
    (df_pitcher['TaggedPitchType'].str.contains('Fastball|FourSeamFastBall|TwoSeamFastBall|OneSeamFastBall|Sinker')),  # Fastballs
    (df_pitcher['TaggedPitchType'].str.contains('Slider|Cutter|Curveball')),  # Breaking Balls
    (df_pitcher['TaggedPitchType'].str.contains('ChangeUp|Splitter|Knuckleball')),  # Offspeed
]
pitch_type_values = [1, 2, 3]
df_pitcher['PitchBucket'] = np.select(pitch_type_conditions, pitch_type_values, default=4)

handedness_conditions = [
    (df_pitcher['PitcherThrows'] == 'Right'),  # RHP
    (df_pitcher['PitcherThrows'] == 'Left')  # LHP
]
handedness_values = [0, 1]
df_pitcher['PitcherHand'] = np.select(handedness_conditions, handedness_values)

# Function to calculate Location+ based on pitch bucket
def calculate_pitch_location_rating(df, pitch_type):
    cleaned_df = df[['PlateLocHeight', 'PlateLocSide']].dropna()
    if pitch_type in ["Fastball", "FourSeamFastBall", "TwoSeamFastBall", "OneSeamFastBall", "Sinker"]:
        return fb_location(cleaned_df)
    elif pitch_type in ["Slider", "Cutter"]:
        return sl_location(cleaned_df)
    elif pitch_type in ["ChangeUp", "Splitter"]:
        return ch_location(cleaned_df)
    elif pitch_type == "Curveball":
        return cu_location(cleaned_df)
    else:
        return other_location(cleaned_df)

# Function to calculate Stuff+ based on pitch bucket
def calculate_pitch_stuff_rating(df, pitch_type):
    cleaned_df = df[['PitchBucket', 'PitcherHand', 'RelSpeed', 'SpinRate', 'InducedVertBreak', 'HorzBreak', 'SpinAxis', 'RelHeight', 'RelSide', 'Extension', 'SpeedDrop', 'pfxz', 'VertApprAngle', 'HorzApprAngle']].dropna()
    if pitch_type in ["Fastball", "FourSeamFastBall", "TwoSeamFastBall", "OneSeamFastBall", "Sinker"]:
        return fastball_stuff(cleaned_df)
    elif pitch_type in ["Slider", "Curveball", "Cutter"]:
        return breaking_stuff(cleaned_df)
    elif pitch_type in ["ChangeUp", "Splitter", "Knuckleball", "Changeup"]:
        return offspeed_stuff(cleaned_df)
    else:
      return 0.0

# Iterate through each division and pitcher
spot = 0
for division in division_list:
    division_df = df_pitcher[df_pitcher['Level'] == division]

    for pitcher in pitcher_list:
        pitcher_df = division_df[division_df['Pitcher'] == pitcher]
        pitcher_teams = pitcher_df['PitcherTeam'].unique()

        for team in pitcher_teams:
            filter_df = pitcher_df[pitcher_df['PitcherTeam'] == team]
            unique_pitch_types = filter_df[filter_df['TaggedPitchType'] != "Undefined"]['TaggedPitchType'].unique()

            for pitch_type in unique_pitch_types:
                locationRating = calculate_pitch_location_rating(filter_df[filter_df['TaggedPitchType'] == pitch_type], pitch_type)
                stuffRating = calculate_pitch_stuff_rating(filter_df[filter_df['TaggedPitchType'] == pitch_type], pitch_type)

                # Update the Location+ column
                pitcher_trackman.iloc[spot, pitcher_trackman.columns.get_loc('Location+')] = locationRating
                pitcher_trackman.iloc[spot, pitcher_trackman.columns.get_loc('Stuff+')] = stuffRating
                spot += 1


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted with

# ***Following Code Used to Predict xBA, xBABIP, xSLG, xwOBA, xISO* (Batters)**

In [ ]:
import joblib
import numpy as np
import pandas as pd
import statistics
from scipy.stats import norm

# Load models only once
hitModel = joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/HitModels/Hit Model/batting_avg_model5.joblib')
singleModel = joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/HitModels/Hit Type Models/single_model.joblib')
doubleModel = joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/HitModels/Hit Type Models/double_model.joblib')
tripleModel = joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/HitModels/Hit Type Models/triple_model.joblib')
hrModel = joblib.load('/content/drive/Shareddrives/USC Baseball/ML Stuff/SabermetricModels/HitModels/Hit Type Models/HR_model.joblib')

# Define a generic prediction function
def predict_hits(filter_saber: DataFrame, model, prob_multiplier: float, division_factor: float = 1) -> float:
    # Select relevant features
    features = {
        hitModel: ['ExitSpeed', 'Angle', 'PlateLocHeight', 'PlateLocSide', 'ContactPositionX', 'ContactPositionY', 'ContactPositionZ', 'pitchType', 'pitcherHand', 'hitterHand'],
        singleModel: ['ExitSpeed', 'AngleSquared', 'HitTrajectoryXc0', 'HitTrajectoryXc1', 'HitTrajectoryXc2', 'HitTrajectoryXc3', 'HitTrajectoryXc4', 'HitTrajectoryXc5', 'HitTrajectoryXc6', 'HitTrajectoryXc7', 'HitTrajectoryXc8', 'HitTrajectoryYc0', 'HitTrajectoryYc1', 'HitTrajectoryYc2', 'HitTrajectoryYc3', 'HitTrajectoryYc4', 'HitTrajectoryYc5', 'HitTrajectoryYc6', 'HitTrajectoryYc7', 'HitTrajectoryYc8', 'HitTrajectoryZc0', 'HitTrajectoryZc1', 'HitTrajectoryZc2', 'HitTrajectoryZc3', 'HitTrajectoryZc4', 'HitTrajectoryZc5', 'HitTrajectoryZc6', 'HitTrajectoryZc7', 'HitTrajectoryZc8'],
        doubleModel: ['ExitSpeed', 'AngleSquared', 'HitTrajectoryXc0', 'HitTrajectoryXc1', 'HitTrajectoryXc2', 'HitTrajectoryXc3', 'HitTrajectoryXc4', 'HitTrajectoryXc5', 'HitTrajectoryXc6', 'HitTrajectoryXc7', 'HitTrajectoryXc8', 'HitTrajectoryYc0', 'HitTrajectoryYc1', 'HitTrajectoryYc2', 'HitTrajectoryYc3', 'HitTrajectoryYc4', 'HitTrajectoryYc5', 'HitTrajectoryYc6', 'HitTrajectoryYc7', 'HitTrajectoryYc8', 'HitTrajectoryZc0', 'HitTrajectoryZc1', 'HitTrajectoryZc2', 'HitTrajectoryZc3', 'HitTrajectoryZc4', 'HitTrajectoryZc5', 'HitTrajectoryZc6', 'HitTrajectoryZc7', 'HitTrajectoryZc8'],
        tripleModel: ['ExitSpeed', 'AngleSquared', 'HitTrajectoryXc0', 'HitTrajectoryXc1', 'HitTrajectoryXc2', 'HitTrajectoryXc3', 'HitTrajectoryXc4', 'HitTrajectoryXc5', 'HitTrajectoryXc6', 'HitTrajectoryXc7', 'HitTrajectoryXc8', 'HitTrajectoryYc0', 'HitTrajectoryYc1', 'HitTrajectoryYc2', 'HitTrajectoryYc3', 'HitTrajectoryYc4', 'HitTrajectoryYc5', 'HitTrajectoryYc6', 'HitTrajectoryYc7', 'HitTrajectoryYc8', 'HitTrajectoryZc0', 'HitTrajectoryZc1', 'HitTrajectoryZc2', 'HitTrajectoryZc3', 'HitTrajectoryZc4', 'HitTrajectoryZc5', 'HitTrajectoryZc6', 'HitTrajectoryZc7', 'HitTrajectoryZc8'],
        hrModel: ['ExitSpeed', 'AngleSquared', 'HitTrajectoryXc0', 'HitTrajectoryXc1', 'HitTrajectoryXc2', 'HitTrajectoryXc3', 'HitTrajectoryXc4', 'HitTrajectoryXc5', 'HitTrajectoryXc6', 'HitTrajectoryXc7', 'HitTrajectoryXc8', 'HitTrajectoryYc0', 'HitTrajectoryYc1', 'HitTrajectoryYc2', 'HitTrajectoryYc3', 'HitTrajectoryYc4', 'HitTrajectoryYc5', 'HitTrajectoryYc6', 'HitTrajectoryYc7', 'HitTrajectoryYc8', 'HitTrajectoryZc0', 'HitTrajectoryZc1', 'HitTrajectoryZc2', 'HitTrajectoryZc3', 'HitTrajectoryZc4', 'HitTrajectoryZc5', 'HitTrajectoryZc6', 'HitTrajectoryZc7', 'HitTrajectoryZc8']
    }
    saberdf = filter_saber[features[model]]

    # Apply model to dataset
    probs = model.predict_proba(saberdf)[:, 1]  # Get probability of the positive class

    # Calculate predicted hits
    num_hits = np.sum(probs * prob_multiplier / division_factor)
    return num_hits

def predHits(filter_saber: DataFrame) -> float:
    return predict_hits(filter_saber, hitModel, prob_multiplier=0.78)

def predSingles(filter_saber: DataFrame) -> float:
    return predict_hits(filter_saber, singleModel, prob_multiplier=0.9368)

def predDoubles(filter_saber: DataFrame) -> float:
    return predict_hits(filter_saber, doubleModel, prob_multiplier=0.835, division_factor=4)

def predTriples(filter_saber: DataFrame) -> float:
    return predict_hits(filter_saber, tripleModel, prob_multiplier=0.8667, division_factor=9)

def predHRs(filter_saber: DataFrame) -> float:
    return predict_hits(filter_saber, hrModel, prob_multiplier=0.92857, division_factor=16)


In [ ]:
import numpy as np
import pandas as pd

# Assuming df3, division_list, batter_list, and batter_trackman are predefined
df3['AngleSquared'] = df3['Angle'] ** 2

# Create hitterHand and pitcherHand columns
df3['hitterHand'] = np.where(df3['BatterSide'] == 'Right', 0, 1)
df3['pitcherHand'] = np.where(df3['PitcherThrows'] == 'Right', 0, 1)

# Create pitchType column
pitch_type_conditions = [
    df3['TaggedPitchType'].str.contains('Fastball|FourSeamFastBall|TwoSeamFastBall|OneSeamFastBall|Sinker'),
    df3['TaggedPitchType'].str.contains('Slider|Cutter'),
    df3['TaggedPitchType'].str.contains('ChangeUp|Splitter'),
    df3['TaggedPitchType'].str.contains('Curveball')
]
pitch_type_values = [0, 1, 2, 3]
df3['pitchType'] = np.select(pitch_type_conditions, pitch_type_values, default=4)

# Clean dataframe by dropping rows with missing values in specific columns
saber_columns_to_check = [
    'ExitSpeed', 'Angle', 'PlateLocHeight', 'PlateLocSide', 'ContactPositionX', 'ContactPositionY',
    'ContactPositionZ', 'pitchType', 'pitcherHand', 'hitterHand', 'HitTrajectoryXc0', 'HitTrajectoryXc1',
    'HitTrajectoryXc2', 'HitTrajectoryXc3', 'HitTrajectoryXc4', 'HitTrajectoryXc5', 'HitTrajectoryXc6',
    'HitTrajectoryXc7', 'HitTrajectoryXc8', 'HitTrajectoryYc0', 'HitTrajectoryYc1', 'HitTrajectoryYc2',
    'HitTrajectoryYc3', 'HitTrajectoryYc4', 'HitTrajectoryYc5', 'HitTrajectoryYc6', 'HitTrajectoryYc7',
    'HitTrajectoryYc8', 'HitTrajectoryZc0', 'HitTrajectoryZc1', 'HitTrajectoryZc2', 'HitTrajectoryZc3',
    'HitTrajectoryZc4', 'HitTrajectoryZc5', 'HitTrajectoryZc6', 'HitTrajectoryZc7', 'HitTrajectoryZc8'
]
hit_columns_to_check = [
    'ExitSpeed', 'Angle', 'PlateLocHeight', 'PlateLocSide', 'ContactPositionX', 'ContactPositionY',
    'ContactPositionZ', 'pitchType', 'pitcherHand', 'hitterHand'
]

SABERcleandf = df3.dropna(subset=saber_columns_to_check)
hitCleanDF = df3.dropna(subset=hit_columns_to_check)
Kdf = df3[df3['KorBB'] == "Strikeout"]
BBdf = df3[df3['KorBB'] == "Walk"]
HBPdf = df3[df3['PitchCall'] == "HitByPitch"]

# Pre-cache column indices for batter_trackman
xBA_idx = batter_trackman.columns.get_loc('xBA')
xSLG_idx = batter_trackman.columns.get_loc('xSLG')
xwOBA_idx = batter_trackman.columns.get_loc('xwOBA')
xISO_idx = batter_trackman.columns.get_loc('xISO')
xBABIP_idx = batter_trackman.columns.get_loc('xBABIP')

spot = 0
for division in division_list:
    division_saber = SABERcleandf[SABERcleandf['Level'] == division]
    division_Kdf = Kdf[Kdf['Level'] == division]
    division_BBdf = BBdf[BBdf['Level'] == division]
    division_HBPdf = HBPdf[HBPdf['Level'] == division]

    for batter in batter_list:
        batter_saber = division_saber[division_saber['Batter'] == batter]
        batter_K = division_Kdf[division_Kdf['Batter'] == batter]
        batter_BB = division_BBdf[division_BBdf['Batter'] == batter]
        batter_HBP = division_HBPdf[division_HBPdf['Batter'] == batter]

        batter_teams = np.array(
            list(set(batter_saber['BatterTeam'].unique()) |
                 set(batter_K['BatterTeam'].unique()) |
                 set(batter_BB['BatterTeam'].unique()) |
                 set(batter_HBP['BatterTeam'].unique()))
        )
        batter_teams_list = batter_teams.tolist()

        for team in batter_teams_list:
            filter_saber = batter_saber[batter_saber['BatterTeam'] == team]
            filter_K = batter_K[batter_K['BatterTeam'] == team]
            filter_BB = batter_BB[batter_BB['BatterTeam'] == team]
            filter_HBP = batter_HBP[batter_HBP['BatterTeam'] == team]

            saberSize = filter_saber.shape[0]
            kSize = filter_K.shape[0]
            bbSize = filter_BB.shape[0]
            hbpSize = filter_HBP.shape[0]
            totalSize = saberSize + kSize + bbSize + hbpSize

            if totalSize < 20:
                batter_trackman.iloc[spot, [xBA_idx, xSLG_idx, xwOBA_idx, xISO_idx]] = ""
            else:
                numHits = predHits(filter_saber)
                numSingles = predSingles(filter_saber)
                numDoubles = predDoubles(filter_saber)
                numTriples = predTriples(filter_saber)
                numHRs = predHRs(filter_saber)
                numKs = kSize
                numBBs = bbSize
                numHBPs = hbpSize
                numBIPs = saberSize
                numABs = numBIPs + numKs
                numPAs = numABs + numBBs

                numTotalBases = numSingles + (2 * numDoubles) + (3 * numTriples) + (4 * numHRs)
                xBA = numHits / ((0.94 * numBIPs) + (0.552 * numKs))
                xSLG = numTotalBases / ((0.552 * numKs) + numBIPs)
                xwOBA = ((0.690 * (0.552 * numBBs)) + (0.722 * (0.552 * numHBPs)) + (0.888 * numSingles) +
                         (1.271 * numDoubles) + (1.616 * numTriples) + (2.101 * numHRs)) / (
                                numBIPs + (0.552 * numKs) + (0.552 * numBBs) + (0.552 * numHBPs))
                xISO = xSLG - xBA

                batter_trackman.iloc[spot, [xBA_idx, xSLG_idx, xwOBA_idx, xISO_idx]] = [xBA, xSLG, xwOBA, xISO]

            if saberSize < 20:
                batter_trackman.iloc[spot, xBABIP_idx] = ""
            else:
                xBABIP = numHits / (0.94 * saberSize)
                batter_trackman.iloc[spot, xBABIP_idx] = xBABIP

            spot += 1


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted with

# Exports dataframes to csvs

In [ ]:
# Check for missing values in the 'Whiff %' column
batter_trackman = batter_trackman.dropna(subset=['Whiff %'])
# Filters out the dataset to only contain players with over 20 BIP
# batter_trackman = batter_trackman[batter_trackman['Sample Size'] >= 20]

In [ ]:
# makes a csv of transfer pitcher trackman
csv = pitcher_trackman.to_csv("/content/drive/Shareddrives/USC Baseball/Transfer Portal Information/2024 Transfer Portal/transfer_pitcher2024Trackman.csv", index=False)


# makes a csv of transfer batter trackman
csv = batter_trackman.to_csv("/content/drive/Shareddrives/USC Baseball/Transfer Portal Information/2024 Transfer Portal/transfer_batter2024Trackman.csv", index=False)

Individual Hitter or Pitcher Sheet Exportations

In [ ]:
# makes a csv of transfer pitcher trackman
# csv = pitcher_trackman.to_csv("/content/drive/Shareddrives/USC Baseball/Transfer Portal Information/2024 Transfer Portal/transfer_pitcher2024Trackman.csv", index=False)

In [ ]:
# makes a csv of transfer batter trackman
# csv = batter_trackman.to_csv("/content/drive/Shareddrives/USC Baseball/Transfer Portal Information/2024 Transfer Portal/transfer_batter2024Trackman.csv", index=False)